#### Homework (migrate sqlite -> postgres, sql requests)
Skvorsov Kirill (M8O-108M-22)

Дана следующая база данных: https://www.kaggle.com/datasets/davidcochran/formula-1-race-data-sqlite

Задача: выгрузить файлы, положить в БД (в данном случае файлы для sqlite, но есть способы положить это дело в PostgreSQL: https://hevodata.com/learn/sqlite-to-postgresql/)

Конкретных заданий, как были в с самолетами расписывать не буду, но скажу лишь о том, что я бы хотел увидеть в результате вашей работы:

1. Наличие всех возможных запросов, которые мы с вами обсуждали (проекция, выборка, объединение, пересечение, и пр.). Вроде их получается 8 шт
2. Наличие всех возможных вариантов выполнения тех или иных запросов (т.е. постараться использовать все директивы, от простых WHERE, то сложных OUTER JOIN, RIGHT JOIN и пр.) Как минимум постараться на запрос сделать 2 штуки, там где это возможно. (случаи с простой выборкой WHERE понятно дело требовать там больше 1 запроса смысла нет)
3. По формату оформления - было бы неплохо чтобы вы в итоге дали какой-то отчет, не важно в каком виде - docx, excel, ipynb или что-то в чем вам было бы удобнее - главное объяснить какие вы использовали таблицы, какая семантика у данных, с которыми вы работали, чтобы было понятно, зачем тот или иной запрос тут нужен. Достаточно 1-2 предложений. 



Миграцию пытался сделать через pgloader, но некоторые данные были немного кривые
(ERROR Database error 42601: zero-length delimited identifier at or near """")

Поэтому пришлось переносить данные руками (скриптом). 

Переливать через джанго (как показано в статье) не получилось в силу ошибки на стадии обратной загрузки.
(опять же возможно в силу немного кривых данных или рук :))

In [1]:
# Создадим папку куда сложим все csv файлы
import os

folder_name = "csv_sqlite"
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

In [2]:
# Загрузим базы из sqlite в csv 
import sqlite3

import pandas as pd 

con = sqlite3.connect("Formula1.sqlite")

cursor = con.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables_list = cursor.fetchall()

for table_name in tables_list:
    table_name = table_name[0]
    
    data = []
    for row in cursor.execute(f"SELECT * from {table_name};"):        
        """
        Этот хардкод нужен для исправления ошибок.
        Какие таблички имеем такие имеем.
        """
        converted_row = list(row)
        if table_name == 'qualifying' or table_name == 'races':
            for idx, val in enumerate(converted_row):
                if val == '\x00':
                    converted_row[idx] = ''
        
        data.append(tuple(converted_row))
    
    columns = [disc[0] for disc in cursor.description]
    
    df = pd.DataFrame(data, columns=columns)
    
    # убираем лишние колонки, иначе переливание из csv будет выкидывть ошибку
    for column in columns:
        if column == '':
            df = df.drop(columns=[column])
    
    path = os.path.join(folder_name, table_name + '.csv')
    try:
        df.to_csv(path, index=False)
    except (Exception) as e:
        print(f'Err: {e} on {table_name}')
         
con.close()

pgfutter (https://github.com/lukasmartinelli/pgfutter/tree/master) отказался работать.

Руками создаем таблички и переливаем в них данные из csv.
(`\copy circuits from '/Users/skvortsovkirill/magistracy/data_analyze/hw_db/csv_sqlite/circuits.csv' delimiter ',' csv header;` и так много раз)

#### Запросы
Для читаемости результатов поставил limit

In [3]:
from sqlalchemy import create_engine, text

engine = create_engine("postgresql:///homework")

with engine.connect() as conn:
    conn = conn.execution_options(isolation_level="REPEATABLE READ")
    
    with conn.begin():
        # Сырые запросы 
        """
    1. Проекция
        Выбрать страны и названия стран из таблицы circuits.
        """
        request =\
        """
        select 
            name, country 
        
        from 
            circuits 
        
        limit 5;
        """
        for row in conn.execute(request):
            print(row)
    
        print('-' * 30)

        """
    2. Выборка
        Выбрать испанские трассы.
        """
        request =\
        """
        select 
            * 
        
        from 
            circuits 
        
        where 
            country = 'Russia'

        limit 5;
        """
        for row in conn.execute(request):
            print(row)
        
        print('-' * 30)
        
        """
    3. Соединение
        Вывести названия трасс и года, в которых были гонки.
        """
        
        request =\
        """
        select 
            circuits.name, year 
        
        from 
            circuits inner join races using(circuitId) 
        
        order by
            name, year

        limit 5;
        """
        
        for row in conn.execute(request):
            print(row)
        
        print('-' * 30)

        """
    4. Объединение
        Немного сложно придумать реальный полезный запрос, поэтому синтетика.
        Вывести все трассы из стран начинающихся на A и на M.
        """
        request =\
        """
        select 
            name as name_for_union
        
        from 
            circuits
        
        where
            name like 'A%'
        
        union
        
        select 
            name as name_for_union
        
        from 
            circuits
        
        where
            name like 'M%'
        
        limit 5;
        """
        
        for row in conn.execute(text(request)):
            print(row)
        
        print('-' * 30)
        
        """
    5. Пересечение (возможно сделать через intersect)
        Вывести топ 7 водителей по результатам гонок. 
        """
        request =\
        """
        select 
            t.forename, t.surname
        
        from
            (
                select
                    forename, surname, sum(points) as total
                
                from
                    drivers inner join driver_standings using(driverId)
                    
                group by
                    forename, surname
            ) as t
        
        order by
            total desc
        
        
        limit 7;
        """
        
        for row in conn.execute(text(request)):
            print(row)
        
        print('-' * 30)
        
        
        """
    6. Разность
        Вывести водителей, для которых нет результатов гонок.
        """
        request =\
        """
        select 
            forename, surname
        
        from
            drivers
            
        where
            driverId in (
                select driverId from drivers
                except all
                select driverId from driver_standings
            )

        limit 5;
        """
        
        for row in conn.execute(text(request)):
            print(row)
        
        print('-' * 30)
        
        """
    7. Декартово произведение
        Запрос придумать еще сложнее т.к. напрямую cross join не используется,
        а обычно является результатом для фильтрации при использовании where.
        Перепишем (3) запрос через cross join.
        """
        request =\
        """
        select 
            circuits.name, races.year 
        
        from 
            circuits cross join races
        
        where 
            circuits.circuitId = races.circuitId
        
        order by
            name, year

        limit 5;
        """
        
        for row in conn.execute(text(request)):
            print(row)
        
        print('-' * 30)

('Albert Park Grand Prix Circuit', 'Australia')
('Sepang International Circuit', 'Malaysia')
('Bahrain International Circuit', 'Bahrain')
('Circuit de Barcelona-Catalunya', 'Spain')
('Istanbul Park', 'Turkey')
------------------------------
(71, 'sochi', 'Sochi Autodrom', 'Sochi', 'Russia', Decimal('43.405700'), Decimal('39.957800'), None, 'http://en.wikipedia.org/wiki/Sochi_Autodrom')
------------------------------
('A1-Ring', 1970)
('A1-Ring', 1971)
('A1-Ring', 1972)
('A1-Ring', 1973)
('A1-Ring', 1974)
------------------------------
('AutÃ_dromo Hermanos RodrÃ_guez',)
('Albert Park Grand Prix Circuit',)
('Marina Bay Street Circuit',)
('AutÃ_dromo do Estoril',)
('Autodromo Nazionale di Monza',)
------------------------------
('Lewis', 'Hamilton')
('Sebastian', 'Vettel')
('Fernando', 'Alonso')
('Nico', 'Rosberg')
('Kimi', 'R�_ikk̦nen')
('Michael', 'Schumacher')
('Jenson', 'Button')
------------------------------
('Markus', 'Winkelhock')
('St̩phane', 'Sarrazin')
('Vincenzo', 'Sospiri')


In [5]:
# Маппинг на схему из алхимии 
"""
Отсортировать водителей в порядке убывания по среднему времени затраченному на питстоп
"""
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, relationship
from sqlalchemy.sql import func
from sqlalchemy import select
from sqlalchemy import Column, String, Integer, ForeignKey, Interval

Base = automap_base()

class Drivers(Base):
    __tablename__ = 'drivers'
    
    driver_id = Column('driverid', Integer, primary_key=True)
    driver_ref = Column('driverref', String)
    number = Column('number', String)
    code = Column('code', String)
    forename = Column('forename', String)
    surname = Column('surname', String)
    dob = Column('dob', String)
    nationality = Column('nationality', String)
    url = Column('url', String)
 
    
class Pitstops(Base):
    __tablename__ = 'pitstops'
    
    race_id = Column('raceid', Integer, ForeignKey('races.raceid'), primary_key=True)
    driver_id = Column('driverid', Integer, ForeignKey('drivers.driverid'))
    stop = Column('stop', Integer)
    time = Column('time', String)
    duration = Column('duration', Interval)
    milliseconds = Column('milliseconds', Integer)
    

Base.prepare(autoload_with=engine)

with Session(engine) as session:
    query = select(
            [
                Drivers.surname, 
                Drivers.forename, 
                func.avg(Pitstops.milliseconds).label('avg_time')
            ]
        ).filter(
            Drivers.driver_id == Pitstops.driver_id
        ).group_by(
            Drivers.surname, Drivers.forename
        ).order_by(
            func.avg(Pitstops.milliseconds).desc()
        ).limit(9)

    for res in session.execute(query):
        print(res.surname, res.forename, round(res.avg_time, 3))


Ocon Esteban 126838.596
Wehrlein Pascal 101109.321
Nasr Felipe 96744.693
Magnussen Kevin 79356.427
Sainz Carlos 78247.257
Verstappen Max 71321.288
Guti̩rrez Esteban 66596.636
Bottas Valtteri 64877.893
Kvyat Daniil 63186.403


Доказательство, что бд postgres, а не отредактированный jupyter + sqlite

<img src="1.png" width="300" height="150">
<img src="2.png" width="600" height="300">

